1. Definir textos a serem vetorizados

In [12]:
db_path = "./DiscursosSenadores_02_05_2025_analisado.sqlite"


import sqlite3
import pandas as pd


conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query para encontrar os códigos dos pronunciamentos
cursor.execute("""
    SELECT CodigoPronunciamento
    FROM Discursos
    WHERE LOWER(TextoIntegral) LIKE '%constitui%'
""")
codigos_pronunciamentos = [row[0] for row in cursor.fetchall()]

# Query para obter os dados desejados
query = f"""
    SELECT
        d.CodigoPronunciamento,
        d.Indexacao,
        da.SumarioConstituicao,
        da.TrechosConstituicao,
        da.NormPredicacao,
        da.NormImplicacao,
        da.NormConclusao,
        da.NormTrecho,
        da.AvalPredicacao,
        da.AvalImplicacao,
        da.AvalConclusao,
        da.AvalTrecho
    FROM Discursos d
    JOIN AnaliseCorpusTodo da ON d.CodigoPronunciamento = da.CodigoPronunciamento
    WHERE d.CodigoPronunciamento IN ({','.join(['?'] * len(codigos_pronunciamentos))})
"""

cursor.execute(query, codigos_pronunciamentos)
results = cursor.fetchall()

# Criar um DataFrame do Pandas
df = pd.DataFrame(results, columns=['CodigoPronunciamento', 'Indexacao', 'SumarioConstituicao', 'TrechosConstituicao', 'NormPredicacao', 'NormImplicacao', 'NormConclusao', 'NormTrecho', 'AvalPredicacao', 'AvalImplicacao', 'AvalConclusao', 'AvalTrecho'])

conn.close()



2. Fazer os embeddings

In [ ]:
!pip install faiss-cpu openai
import time
import random
import numpy as np
import faiss
import os
from openai import OpenAI
# Configurar a API da OpenAI
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

# Definir a pasta onde os arquivos FAISS serão salvos
faiss_pasta = "../data/discursos/embeddings"

# Criar a pasta se não existir
os.makedirs(faiss_pasta, exist_ok=True)

# Definir as colunas de texto para gerar embeddings
colunas_embeddings = ['Indexacao', 'SumarioConstituicao', 'TrechosConstituicao', 'NormPredicacao', 'NormImplicacao', 'NormConclusao', 'NormTrecho', 'AvalPredicacao', 'AvalImplicacao', 'AvalConclusao', 'AvalTrecho']

# Parâmetros do FAISS
embedding_dim = 3072  # Ajuste conforme o modelo
lote_tamanho = 100  # Número de embeddings processados antes de salvar no FAISS
checkpoint_tamanho = 500  # Checkpoint para salvar o progresso

def carregar_codigos_existentes(nome_arquivo):
    """Carrega os códigos já processados para evitar duplicação."""
    caminho_codigos = os.path.join(faiss_pasta, nome_arquivo)
    if os.path.exists(caminho_codigos):
        return set(np.load(caminho_codigos))  # Convertendo para set para busca rápida
    return set()

def criar_ou_carregar_faiss(nome_arquivo):
    """Cria um índice FAISS ou carrega um existente."""
    caminho_faiss = os.path.join(faiss_pasta, nome_arquivo)
    try:
        index = faiss.read_index(caminho_faiss)
        print(f"✅ Índice FAISS carregado de '{caminho_faiss}'.")
    except:
        index = faiss.IndexFlatL2(embedding_dim)
        print(f"📂 Criando novo índice FAISS '{caminho_faiss}'.")
    return index, caminho_faiss

def salvar_progresso(faiss_index, caminho_faiss, codigos_existentes, caminho_codigos):
    """Salva o índice FAISS e atualiza os códigos já processados."""
    faiss.write_index(faiss_index, caminho_faiss)
    np.save(caminho_codigos, np.array(list(codigos_existentes), dtype=np.int32))
    print(f"💾 Progresso salvo! Índice '{caminho_faiss}' atualizado.")

def gerar_embedding(texto):
    """Gera um embedding usando a API da OpenAI."""
    response = client.embeddings.create(input=texto, model="text-embedding-3-large")
    return np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)

def processar_embeddings(df, coluna, lote_tamanho=100, checkpoint_tamanho=500):
    """
    Processa os textos de uma coluna do DataFrame, gera embeddings e armazena em FAISS.

    Args:
        df (pd.DataFrame): DataFrame contendo os discursos.
        coluna (str): Nome da coluna a ser processada.
        lote_tamanho (int): Quantidade de embeddings processados antes de salvar no FAISS.
        checkpoint_tamanho (int): Quantidade de discursos processados antes de salvar o progresso.

    Returns:
        None
    """
    nome_faiss = f"discursos_{coluna}.index"
    nome_codigos = f"codigos_{coluna}.npy"

    faiss_index, caminho_faiss = criar_ou_carregar_faiss(nome_faiss)
    caminho_codigos = os.path.join(faiss_pasta, nome_codigos)

    # Carregar códigos já processados para evitar duplicação
    codigos_existentes = carregar_codigos_existentes(nome_codigos)
    buffer = []
    codigos_novos = []
    total_processados = 0  # Contador para checkpoints

    for i, row in df.iterrows():
        codigo = int(row["CodigoPronunciamento"])
        texto = str(row[coluna]).strip()  # Converter para string e remover espaços extras

        if not texto:
            print(f"⚠️ Texto vazio para CodigoPronunciamento {codigo}. Pulando...")
            continue

        if codigo in codigos_existentes:
            print(f"🔄 {coluna}: CodigoPronunciamento {codigo} já processado. Pulando...")
            continue

        try:
            embedding = gerar_embedding(texto)
            buffer.append(embedding)
            codigos_novos.append(codigo)
            print(f"[{i+1}/{len(df)}] {coluna}: Texto {codigo} processado.")

            # Adicionar ao FAISS quando atingir o tamanho do lote
            if len(buffer) >= lote_tamanho:
                batch = np.vstack(buffer)
                faiss_index.add(batch)
                buffer.clear()
                total_processados += lote_tamanho
                print(f"✅ {lote_tamanho} embeddings adicionados ao FAISS.")

            # Salvar progresso em checkpoints regulares
            if total_processados >= checkpoint_tamanho:
                codigos_existentes.update(codigos_novos)
                salvar_progresso(faiss_index, caminho_faiss, codigos_existentes, caminho_codigos)
                codigos_novos.clear()
                total_processados = 0  # Reset do contador de checkpoint

        except Exception as e:
            print(f"❌ Erro ao processar CodigoPronunciamento {codigo}: {str(e)}")

        # Pequena pausa para evitar sobrecarga da API
        time.sleep(random.uniform(0.5, 1.5))

    # Salvar os embeddings restantes no FAISS
    if buffer:
        batch = np.vstack(buffer)
        faiss_index.add(batch)
        print("✅ Buffer final de embeddings salvo no FAISS.")

    # Salvar o progresso final
    codigos_existentes.update(codigos_novos)
    salvar_progresso(faiss_index, caminho_faiss, codigos_existentes, caminho_codigos)

    print(f"🚀 Processamento concluído para '{coluna}'!")

# Processar embeddings para cada coluna
for coluna in colunas_embeddings:
    processar_embeddings(df, coluna, lote_tamanho, checkpoint_tamanho)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✅ Índice FAISS carregado de '../data/discursos/embeddings/discursos_Indexacao.index'.
🔄 Indexacao: CodigoPronunciamento 366329 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366331 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366333 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366336 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366337 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366400 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366406 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366407 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366410 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366419 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366421 já processado. Pulando...
🔄 Indexacao: CodigoPronunciamento 366425 já processado. Pulando.